In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df_SDM = pd.read_csv('Web Datasets/VAC14_SDM_output.csv', index_col=False)
df_barry = pd.read_csv('Output Datasets/VAC14_ref_set_all_binary.csv', index_col=False)
print(df_SDM.head(7), '\n')
print(df_barry.head(7))

   PDB file Chain ID variant WT_SSE  WT_RSA(%)  WT_DEPTH (angstrom)  WT_OSP  \
0  1D5R.pdb        A   A121E      E        0.7                12.30   0.510   
1  1D5R.pdb        A   A121P      E        0.7                12.30   0.510   
2  1D5R.pdb        A   K125P      a       21.0                 4.08   0.296   
3  1D5R.pdb        A   A126D      a       55.7                 3.10   0.230   
4  1D5R.pdb        A   A151P      H        0.0                 7.60   0.620   
5  1D5R.pdb        A    A34D      E        0.0                 9.70   0.550   
6  1D5R.pdb        A    A34P      E        0.0                 9.70   0.550   

   WT_SS  WT_SN  WT_SO MT_SSE  MT_RSA(%)  MT_DEPTH (angstrom)  MT_OSP  MT_SS  \
0      0      0      0      E   0.000000                10.70   0.580      1   
1      0      0      0      E   0.000000                10.90   0.570      0   
2      1      0      0      a  17.299999                 4.56   0.375      0   
3      0      0      0      a  68.700000       

In [3]:
df_ref_set = df_barry.drop(['ClinVar','ClinVar significance','COSMIC',
                           'Tumor Type','Allele Frequency','gnomAD'], axis=1)

In [4]:
# listing frame shift and nonsense variants
print(df_ref_set[df_ref_set['variant'].str.contains('X')],'\n')
print(df_ref_set[df_ref_set['variant'].str.contains('fs')])

   variant     vac14      fig4      vac7      vam3      vam7      ypt7  \
30   E285X -0.030303 -0.068966 -0.010000 -0.258065 -0.125000  0.458333   
49   L139X  0.897260  1.211111  1.200000  1.272727  0.588235  0.769231   
51   L320X  0.801170  0.962617  0.842105  0.938776  0.960784  1.000000   
72   R130X  1.061644  1.177778  1.317647  1.151515  0.862745  1.115385   
77   R335X  1.039216  1.393617  0.979381  1.741935  1.384615  2.103448   
91   Y178X  0.854015  0.933333  1.022989  0.885714  1.096774  1.107143   

       vps30     vps38       Mighell  PolyPhen  ground_truth  
30  0.181818 -0.157895   0.082814179       NaN           NaN  
49  1.050000  1.933333   0.049418425       NaN           NaN  
51  0.440000  0.176471  -0.362479123       NaN           NaN  
72  0.300000  1.333333  -0.974823136       NaN           NaN  
77  0.800000  2.833333       #VALUE!       NaN           NaN  
91  0.652174  0.812500  -0.478011694       NaN           NaN   

   variant     vac14      fig4      va

In [5]:
# dropping f/s and X variants
df_ref_set.drop([30,49,51,72,77,91,10,41], axis=0, inplace=True)

In [6]:
df_all = df_ref_set.merge(df_SDM, on='variant', how='left')

In [7]:
print(len(df_all))

90


In [8]:
df_all[df_all['variant'] == 'A79T']

,variant,vac14,fig4,vac7,vam3,vam7,ypt7,vps30,vps38,Mighell,...,WT_SO,MT_SSE,MT_RSA(%),MT_DEPTH (angstrom),MT_OSP,MT_SS,MT_SN,MT_SO,Predicted ddg,Outcome
8,A79T,0.009091,-0.018987,0.028409,0.034483,-0.144578,-0.03125,-0.285714,-0.3,0.054034306,...,0.0,a,80.2,3.2,0.19,0.0,0.0,0.0,-0.11,Reduced stability


In [9]:
df_all = df_all.drop(df_all[['PDB file','Chain ID', 'WT_SSE', 'MT_SSE', 'Outcome']], axis=1)

In [10]:
df_all[df_all['variant'] == 'A79T']

,variant,vac14,fig4,vac7,vam3,vam7,ypt7,vps30,vps38,Mighell,...,WT_SS,WT_SN,WT_SO,MT_RSA(%),MT_DEPTH (angstrom),MT_OSP,MT_SS,MT_SN,MT_SO,Predicted ddg
8,A79T,0.009091,-0.018987,0.028409,0.034483,-0.144578,-0.03125,-0.285714,-0.3,0.054034306,...,0.0,0.0,0.0,80.2,3.2,0.19,0.0,0.0,0.0,-0.11


# setting up training data

In [11]:
training_df = df_all.dropna().copy()

In [12]:
training_df.reset_index(drop=True)

,variant,vac14,fig4,vac7,vam3,vam7,ypt7,vps30,vps38,Mighell,...,WT_SS,WT_SN,WT_SO,MT_RSA(%),MT_DEPTH (angstrom),MT_OSP,MT_SS,MT_SN,MT_SO,Predicted ddg
0,A79T,0.009091,-0.018987,0.028409,0.034483,-0.144578,-0.031250,-0.285714,-0.300000,0.054034306,...,0.0,0.0,0.0,80.2,3.2,0.19,0.0,0.0,0.0,-0.11
1,C124S,1.040698,1.191304,1.117647,2.069767,1.101695,3.230769,1.500000,1.285714,-1.223910727,...,1.0,1.0,1.0,4.3,7.3,0.54,1.0,1.0,0.0,-1.44
2,C211W,0.176471,0.221154,0.403846,0.000000,0.324324,0.307692,0.360000,0.500000,-0.250662359,...,0.0,0.0,1.0,0.0,7.6,0.69,1.0,0.0,0.0,-1.53
3,D107V,0.968153,1.336957,1.100840,1.431818,0.952381,2.312500,2.842105,2.192308,-0.712965385,...,1.0,0.0,0.0,41.6,3.8,0.43,0.0,0.0,0.0,0.72
4,D252G,0.581699,0.957447,1.061856,1.806452,0.769231,1.965517,1.866667,2.388889,-0.522853504,...,1.0,1.0,0.0,27.2,5.5,0.31,0.0,0.0,0.0,-0.72
5,D268E,0.019324,0.023622,0.039474,0.046875,-0.066667,0.750000,-0.058824,0.085714,0.056335824,...,0.0,0.0,0.0,51.0,3.5,0.28,0.0,0.0,0.0,1.16
6,F241S,0.660131,1.191489,1.247423,1.612903,1.134615,1.896552,1.533333,1.833333,-0.599858557,...,0.0,0.0,0.0,18.1,6.9,0.40,0.0,0.0,0.0,-2.96
7,F56C,0.647059,0.884615,1.033898,1.166667,1.320755,1.612903,4.857143,4.833333,-0.240570479,...,0.0,0.0,0.0,0.5,6.6,0.48,0.0,0.0,0.0,-0.93
8,G127R,0.971264,0.957627,1.007299,1.250000,1.015873,1.444444,0.904762,2.538462,-0.55825153,...,0.0,0.0,0.0,0.0,8.3,0.74,1.0,0.0,1.0,-2.85
9,G129E,1.301471,1.288462,1.457627,2.000000,1.886792,1.677419,5.642857,6.250000,-1.352440879,...,0.0,0.0,0.0,4.5,7.3,0.61,1.0,1.0,0.0,1.95


In [13]:
# how many variants with ground_truth=0?
print(training_df[training_df[['ground_truth']] == 0].count()['ground_truth'])
print()
# how many entries have ground truth?
print(training_df.count()['ground_truth'])

8

38


In [14]:
y = training_df[['ground_truth']]
print(y.shape)

(38, 1)


In [15]:
def do_sentinel(sen):
    X = training_df.drop(['variant','ground_truth'], axis=1)
    X.drop(X.iloc[:,:10], axis=1, inplace=True)
    X.insert(0, sen, training_df[sen])
    return X

In [26]:
#X = do_sentinel('vac14')
X = do_sentinel('fig4')
#X = do_sentinel('vac7')
#X = do_sentinel('vam3')
#X = do_sentinel('vam7')
#X = do_sentinel('ypt7')
#X = do_sentinel('vps30')
#X = do_sentinel('vps38')
#X = do_sentinel('Mighell')
#X = do_sentinel('PolyPhen')

In [27]:
X.head()

,fig4,WT_RSA(%),WT_DEPTH (angstrom),WT_OSP,WT_SS,WT_SN,WT_SO,MT_RSA(%),MT_DEPTH (angstrom),MT_OSP,MT_SS,MT_SN,MT_SO,Predicted ddg
8,-0.018987,86.3,3.1,0.19,0.0,0.0,0.0,80.2,3.2,0.19,0.0,0.0,0.0,-0.11
9,1.191304,4.3,6.7,0.52,1.0,1.0,1.0,4.3,7.3,0.54,1.0,1.0,0.0,-1.44
10,0.221154,0.3,7.3,0.50,0.0,0.0,1.0,0.0,7.6,0.69,1.0,0.0,0.0,-1.53
12,1.336957,32.0,3.8,0.44,1.0,0.0,0.0,41.6,3.8,0.43,0.0,0.0,0.0,0.72
14,0.957447,8.6,5.7,0.39,1.0,1.0,0.0,27.2,5.5,0.31,0.0,0.0,0.0,-0.72


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=0)

# Apply logistic regression

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [36]:
LR = LogisticRegression(random_state=0, solver='liblinear').fit(
    X_train, y_train.values.ravel()) 

In [37]:
LR_pred = LR.predict(X_test)

In [38]:
print(LR_pred)

[0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1.]


In [39]:
print('The accuracy is {0:0.3f}'.format(accuracy_score(y_test, LR_pred)))

The accuracy is 0.833


In [40]:
LR_kfold_scores = cross_val_score(LR, X, y.values.ravel(), cv=5)
print(LR_kfold_scores, '\n')
print('The average cross-validation score is {0:0.3f}'.format(np.mean(LR_kfold_scores)))

[0.75       0.75       0.75       0.71428571 0.85714286] 

The average cross-validation score is 0.764


In [41]:
print(classification_report(y_test, LR_pred))

              precision    recall  f1-score   support

         0.0       0.33      1.00      0.50         1
         1.0       1.00      0.82      0.90        11

   micro avg       0.83      0.83      0.83        12
   macro avg       0.67      0.91      0.70        12
weighted avg       0.94      0.83      0.87        12

